---
## Assessment of greenhouse gas emissions in the portfolio
---

In [1]:
# load imports
import pandas as pd
import numpy as np
import math
# graphing modules
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# set dataframe options
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [2]:
# execute the helper functions defined for accessing RDP REST API calls 
%run RDPDefines.ipynb

### Load the previously downloaded dataset for climate

In [3]:
allClimate = pd.read_pickle('ClimateData.pkl').astype({
    'OrganizationId': np.int64,
    'FinancialPeriodFiscalYear': int,
    'Scope1EstimatedTotal': float,
    'Scope2EstimatedTotal': float,
    'Scope3EstimatedUpstreamTotal': float,
    'Scope3EstimatedDownstreamTotal': float,
    'Scope1EstimatedMethod': object,
    'Scope2EstimatedMethod': object,
    'Scope3EstimatedUpstreamMethod': object,
    'Scope3EstimatedDownstreamMethod': object,
    'SalesinUSD': float,
    'ExchangeRatetoUSD': float,
    'EnterpriseValueincludingCashandShortTermInvestmentsinUSD': float
})
allClimate

,Scope1EstimatedTotal,Scope3EstimatedDownstreamMethod,Scope3EstimatedUpstreamMethod,Scope1EstimatedMethod,Scope3EstimatedUpstreamTotal,EnterpriseValueincludingCashandShortTermInvestmentsinUSD,ExchangeRatetoUSD,SalesinUSD,Scope2EstimatedTotal,Scope2EstimatedMethod,Scope3EstimatedDownstreamTotal,OrganizationId,FinancialPeriodFiscalYear
0,2897820.690,Aggregated_model,Aggregated_model,Reported_value,151219.350,9937830000.000,0.145,1540440000.000,14637.220,Reported_value,621929.980,4295863771,2022
1,NaN,NaN,NaN,NaN,NaN,NaN,1.210,7867804121.000,NaN,NaN,NaN,4295868688,2014
2,19106.471,Aggregated_model,Aggregated_model,Aggregated_model,10055.910,2669089539.227,0.001,708854835.834,35635.805,Aggregated_model,7342.970,4295881108,2018
3,NaN,NaN,NaN,NaN,NaN,NaN,1.993,5862910000.000,NaN,NaN,NaN,4295895529,2007
4,NaN,NaN,NaN,NaN,NaN,NaN,1.000,289355000.000,NaN,NaN,NaN,4295904372,2008
...,...,...,...,...,...,...,...,...,...,...,...,...,...
114617,121214.000,Aggregated_model,Reported_value,Reported_value,3802.330,5184563521.720,1.000,746044000.000,33674.000,Reported_value,347706.360,4295902499,2019
114618,NaN,NaN,NaN,NaN,NaN,NaN,1.000,15035700000.000,NaN,NaN,NaN,4295903351,2005
114619,1097.434,Aggregated_model,Aggregated_model,Aggregated_model,4872.500,349391504.700,1.000,241426000.000,2381.907,Aggregated_model,6583.320,4295912635,2021
114620,15168.086,Aggregated_model,Aggregated_model,Aggregated_model,48192.820,2148161856.964,0.273,836250000.000,56464.528,Aggregated_model,20482.970,4297883285,2022


### Load the portfolio to be analized
Portfolio is a CSV file with the constituent *RIC* and corresponding *Weight* in percent

*Weight* must add up to 100%

|RIC|WEIGHT %|
|--|--|
|AAPL.O|2.5|
|MSFT.O|1.2|
|..|..|

In [4]:
# load the portfolio to be analized
portfolio = pd.read_csv('Portfolio.csv')
portfolio

,RIC,Weight
0,SMFT3.SA,0.001
1,MGLU3.SA,0.003
2,KBC.BR,0.028
3,ICT.PS,0.006
4,000880.KS,0.002
...,...,...
4158,WEGE3.SA,0.017
4159,8954.T,0.005
4160,SO.N,0.115
4161,YUM.N,0.057


### Convert Symbology

Lookup *Organization PermID* and *Reporting currency* for all instruments

RDP Symbology API: [/data/symbology/beta1/convert]()

In [ ]:
allResponses = []
RIClist = portfolio['RIC'].tolist()
bucketSize = 90

buckets = [ RIClist[i: i + bucketSize] for i in range(0, len(RIClist), bucketSize) ]
for bucket in buckets:
  reqData = {
    "universe": bucket,
    "to": ["OrganizationId"]
  }

  hResp = postRequest('/data/symbology/beta1/convert', reqData)
  print(hResp)
  allResponses = allResponses + hResp['universe']

In [ ]:
# merge Symbology and Currency data into the portfolio
orgsData = pd.DataFrame(allResponses)
portfolio = portfolio.merge(orgsData, how='inner', left_on='RIC', right_on='Instrument')
portfolio = portfolio.drop(['Instrument'], axis = 1)
portfolio['Organization PermID'] = portfolio['Organization PermID'].astype(np.int64)
portfolio

### Define calculation variables

In [7]:
# define the calculation parameters
PortfolioAmountInvested = 1000000
years = [2019, 2020, 2021, 2022]
measures = ['Scope1EstimatedTotal', 'Scope2EstimatedTotal', 'Scope3EstimatedUpstreamTotal', 'Scope3EstimatedDownstreamTotal']

# other parameters used in the notebook
lsegColors = ['#FF5000', '#FFC800', '#00D0D4', '#9064CD', '#00C389', '#FFFF00']

### Calculate: Data coverage % of this portfolio 
---

In [8]:
# calculate the coverage % for each scope for each year for this portfolio
cMmeasures = ['Scope1EstimatedTotal', 'Scope1EstimatedMethod', 'Scope2EstimatedTotal', 'Scope2EstimatedMethod', 'Scope3EstimatedUpstreamTotal', 'Scope3EstimatedUpstreamMethod', 'Scope3EstimatedDownstreamTotal', 'Scope3EstimatedDownstreamMethod', 'SalesinUSD', 'EnterpriseValueincludingCashandShortTermInvestmentsinUSD']

def computeCoverageHoldings(year):
    yPort = portfolio.merge(allClimate[allClimate['FinancialPeriodFiscalYear'] == year], how='left', left_on='Organization PermID', right_on='OrganizationId')
    pSize = len(yPort.index)
    return [((pSize - yPort[measure].isna().sum()) / pSize * 100) for measure in cMmeasures]


def computeCoverageWeight(year):
    yPort = portfolio.merge(allClimate[allClimate['FinancialPeriodFiscalYear'] == year], how='left', left_on='Organization PermID', right_on='OrganizationId')
    return [yPort[yPort[measure].notna() & (yPort[measure] != 0) ]['Weight'].sum() for measure in cMmeasures]


In [9]:
# get data for all the measures for all the years
hoCoData = []
wtCoData = []
for yr in years:
    print(f'Getting coverage data for {yr}')
    hoCoData.append(computeCoverageHoldings(yr))
    wtCoData.append(computeCoverageWeight(yr))


Getting coverage data for 2019
Getting coverage data for 2020
Getting coverage data for 2021
Getting coverage data for 2022


In [24]:
# plot the coverage results
plotData = []
idx = 0
for yr in years:
    plotData.append(go.Bar(x = cMmeasures, name = str(yr),  y = hoCoData[idx],  text = [int(x) for x in hoCoData[idx]], marker_color=lsegColors[idx]))
    idx = idx + 1

layout = go.Layout(barmode = 'group')
fig = go.Figure(data = plotData, layout = layout)
fig.update_layout(title_text="Data coverage (% of portfolio holdings)", bargap=0.3)
fig.show()

plotData = []
idx = 0
for yr in years:
    plotData.append(go.Bar(x = cMmeasures, name = str(yr),  y = wtCoData[idx],  text = [int(x) for x in wtCoData[idx]], marker_color=lsegColors[idx]))
    idx = idx + 1

layout = go.Layout(barmode = 'group')
fig = go.Figure(data = plotData, layout = layout)
fig.update_layout(title_text="Data coverage (% of portfolio weight)", bargap=0.3)
fig.show()

### Calculate: Weighted Average Carbon Intensity
---
### WACI = ${\sum_{Portfolio}Weight \text{ } * \text{ } {GHG \text{ } Emissions \over Revenue} \over \sum_{Portfolio}Weight \text{ } (GHG \text{ } Emissions \text{ } or \text{ } Revenue \neq 0)} $

In [11]:
# define the formula for weighted carbon intesity
#   Sum(weight * estimate/revenue)/Sum(weight) if estimate is not 0
def getWACI(tPort, measure):
    sDF = tPort[tPort[measure].notna() & tPort['SalesinUSD'].notna() & (tPort['SalesinUSD'] != 0)]
    neu = (sDF['Weight'] * sDF[measure] / (sDF['SalesinUSD'] / 1000000)).sum()
    deno = sDF['Weight'].sum()
    return neu/deno


def getCarbonIntensityForYear(year):
    # merge data for a particular year into portfolio
    yPort = portfolio.merge(allClimate[allClimate['FinancialPeriodFiscalYear'] == year], how='left', left_on='Organization PermID', right_on='OrganizationId')
    return [getWACI(yPort, m) for m in measures]

In [12]:
# get data for all the measures for all the years
wciData = []
for yr in years:
    print(f'Getting data for {yr}')
    wciData.append(getCarbonIntensityForYear(yr))

Getting data for 2019
Getting data for 2020
Getting data for 2021
Getting data for 2022


In [25]:
# chart it in a bar graph
plotData = []
idx = 0
for yr in years:
    plotData.append(go.Bar(x = measures, name = str(yr),  y = wciData[idx],  text = [int(x) for x in wciData[idx]], marker_color=lsegColors[idx]))
    idx = idx + 1

layout = go.Layout(barmode = 'group')
fig = go.Figure(data = plotData, layout = layout)
fig.update_layout(title_text="Portfolio's Weighted Average Carbon Intensity (Weight by Scope)", yaxis=dict(title='Tonne CO₂/MUSD sales'))
fig.show()

### Calculate: Financed Emissions
---
### FE = ${\sum_{Portfolio} GHG \text{ } Emissions \text{ } * \text{ } {Constituent \text{ } Weight \text{ } * \text{ } Amount \text{ } Invested \over EVIC} \over \sum_{Portfolio}Weight \text{ } (GHG \text{ } Emissions \text{ } or \text{ } EVIC \neq 0)} $

In [14]:
# define the formula for financed emissions
def getFE(tPort, measure):
    sDF = tPort[tPort[measure].notna() & tPort['EnterpriseValueincludingCashandShortTermInvestmentsinUSD'].notna() & (tPort['EnterpriseValueincludingCashandShortTermInvestmentsinUSD'] != 0)]
    neu = (sDF[measure] * PortfolioAmountInvested * sDF['Weight'] / sDF['EnterpriseValueincludingCashandShortTermInvestmentsinUSD'] ).sum()
    deno = sDF['Weight'].sum()
    return neu/deno


def getFinancedEmissionsForYear(year):
    # merge data for a particular year into portfolio
    yPort = portfolio.merge(allClimate[allClimate['FinancialPeriodFiscalYear'] == year], how='left', left_on='Organization PermID', right_on='OrganizationId')
    return [getFE(yPort, m) for m in measures]


In [15]:
# get data for all the measures for all the years
feData = []
for yr in years:
    print(f'Getting data for {yr}')
    feData.append(getFinancedEmissionsForYear(yr))


Getting data for 2019
Getting data for 2020
Getting data for 2021
Getting data for 2022


In [27]:
# chart it in a bar graph
plotData = []
idx = 0
for yr in years:
    plotData.append(go.Bar(x = measures, name = str(yr),  y = feData[idx],  text = [int(x) for x in feData[idx]], marker_color=lsegColors[idx]))
    idx = idx + 1

layout = go.Layout(barmode = 'group')
fig = go.Figure(data = plotData, layout = layout)
fig.update_layout(title_text=f"Financed Emissions for a portfolio valued at {PortfolioAmountInvested:,}", yaxis=dict(title='Tonne CO₂/MUSD'))
fig.show()

### Calculate: PCAF Data Quality Score
---

### PCAF = $ \% \text{ } of \text{ } Financed \text{ } Emissions \text{ } Score \text{ } * \text{ } {\sum_{Portfolio}GHG \text{ } Emissions \text{ } * \text{ } {{Weight \text{ } * \text{ } Amount \text{ } Invested} \over EVIC} \text{ } * \text{ } (PCAF \text{ } Score \text{ } = \text{ } x)} \over Financed \text{ } Emissions $ 


In [17]:
# assign a score to each type of estimation method
scoreMapping = {
    'Scope1EstimatedMethod': {
        'Reported': 2,
        'Energy_model': 3,
        'Energy_extrapolated': 4,
        'Winsorized': 4,
        'Extrapolated': 4,
        'Aggregated Estimate': 5,
        'Aggregated_model': 5,
        'Reported_value': 2,
    },

    'Scope2EstimatedMethod': {
        'Reported': 2,
        'Winsorized': 4,
        'Extrapolated': 4,
        'Aggregated Estimate': 5,
        'Aggregated_model': 5,
        'Reported_value': 2
    },

    'Scope3EstimatedDownstreamMethod':	{
        'Reported': 2,
        'Fossil_fuel_production_model': 3,
        'Winsorized': 4,
        'Extrapolated': 4,
        'Aggregated_model': 5,
        'Reported_value': 2,
        'Aggregated model': 5,
        'Fossil fuel production model': 3
    },

    'Scope3EstimatedUpstreamMethod': {
        'Reported': 2,
        'Winsorized': 4,
        'Extrapolated': 4,
        'Aggregated_model': 5,
        'Reported_value': 2,
        'Aggregated model': 5
    }
}

In [18]:
# define parameters for the PCAF calculation - year and the fields to be used
pcafYear = 2021
numberScoreGrades = 6
pcafMeasures = {
    'Scope1EstimatedMethod': 'Scope1EstimatedTotal',
	'Scope2EstimatedMethod': 'Scope2EstimatedTotal',
	'Scope3EstimatedUpstreamMethod': 'Scope3EstimatedUpstreamTotal',
	'Scope3EstimatedDownstreamMethod': 'Scope3EstimatedDownstreamTotal'
}

In [19]:
# define the formula for PCAF Score grade
def getPCAFByWeight(tPort, scope):
    scoresByWeight = [0] * numberScoreGrades
    # aggregate all the weights for a particular score
    sDF = tPort[tPort['EnterpriseValueincludingCashandShortTermInvestmentsinUSD'].notna() & (tPort['EnterpriseValueincludingCashandShortTermInvestmentsinUSD'] != 0)]
    for grade in range(numberScoreGrades):
        scoresByWeight[grade] = sDF[sDF[scope] == grade]['Weight'].sum() 

    # also add the weights which don't have enterprise value into the n/a bucket
    sDF = tPort[tPort['EnterpriseValueincludingCashandShortTermInvestmentsinUSD'].isna() | (tPort['EnterpriseValueincludingCashandShortTermInvestmentsinUSD'] == 0)]
    scoresByWeight[0] += sDF['Weight'].sum() 

    return scoresByWeight

In [20]:
# define the formula for PCAF Score grade
def getPCAFByFE(tPort, scope):
    # calculate the financed emissions for a scope
    tPort[f'fe_{scope}'] = tPort[pcafMeasures[scope]] * PortfolioAmountInvested * tPort['Weight'] / tPort['EnterpriseValueincludingCashandShortTermInvestmentsinUSD']
    # find coverage % for this scope
    coverage = tPort[tPort[pcafMeasures[scope]].notna() & tPort['EnterpriseValueincludingCashandShortTermInvestmentsinUSD'].notna()]['Weight'].sum()
    # find the ratio of FE to coverage
    R4170 = tPort[f'fe_{scope}'].sum()/coverage
    # calculate rebased financed emissions and insert into FE misssing values
    tPort['rebasedFE'] = tPort['Weight'] * PortfolioAmountInvested * R4170 / 1000000
    tPort[f'fe_{scope}'] = tPort[f'fe_{scope}'].fillna(tPort['rebasedFE'])
    allFE = tPort[f'fe_{scope}'].sum()
    # generate score grades by FE
    scoresByFE = [ (tPort[tPort[scope] == grade][f'fe_{scope}'].sum()/allFE) * 100 for grade in range(numberScoreGrades)]
    return scoresByFE

In [21]:
# merge data for a particular year into portfolio
pcafPort = portfolio.merge(allClimate[allClimate['FinancialPeriodFiscalYear'] == pcafYear], how='left', left_on='Organization PermID', right_on='OrganizationId')
# replace the method values with score grade numbers 
pcafPort = pcafPort.replace(scoreMapping)
# fill up the unavailable methods with 0
pcafPort.fillna({k:0 for k in pcafMeasures.keys()}, inplace=True)

pcafWeights = [getPCAFByWeight(pcafPort, measure) for measure in pcafMeasures.keys()]
pcafFE = [getPCAFByFE(pcafPort, measure) for measure in pcafMeasures.keys()]

### Pie chart for PCAF Scores

In [28]:
scoreLabels = ['No Data', '1', '2', '3', '4', '5']
measureNames = pcafMeasures.keys()

# Create subplots for index weight pie charts
fig = make_subplots(rows=1, cols=len(measureNames), specs=[[{'type':'domain'}] * len(measureNames)] )
idx = 0
for m in measureNames:
    fig.add_trace(go.Pie(labels=scoreLabels, values=pcafWeights[idx], name=m), 1, 1 + idx)
    idx = idx + 1

fig.update_traces(hole=.5, hoverinfo="label+percent+name", marker=dict(colors=lsegColors))
fig.update_layout(title_text = f"PCAF quality score for {pcafYear} (Index Weight)")
fig.show()

# Create subplots for Financed Emissions pie charts
fig = make_subplots(rows=1, cols=len(measureNames), specs=[[{'type':'domain'}] * len(measureNames)] )
idx = 0
for m in measureNames:
    fig.add_trace(go.Pie(labels=scoreLabels, values=pcafFE[idx], name=m), 1, 1 + idx)
    idx = idx + 1

fig.update_traces(hole=.5, hoverinfo="label+percent+name", marker=dict(colors=lsegColors))
fig.update_layout(title_text = f"PCAF quality score for {pcafYear} (Financed Emissions)")
fig.show()

### One time step: Download and save the Bulk files
---

In [3]:
# For clients with subscription to full climate data
# download the climate standard file:
bFiles = downloadJSONBulkFile('bulk-Climate', 'ContentType:Climate', ['Measures-Full-v1-DataItems', 'Json', 'Init'])

# For clients with subscription to ESG data Climate add-on package
# Download the climate addon file 
#   Bucket: bulk-Climate
#   ContentType: ClimateAddOn
#   FileSetName: Bulk-Climate-Global-Measures-AddOn-v1-DataItems-Jsonl-Init

# Also download the ESG Measures file to get Company Revenue and Enterprise Value
# Finally combine and save the information in a unified DataFrame

Getting access token...
...token received
{'value': [{'id': '4058-6e0c-6da26e1f-ac4c-d24ebbf69496', 'name': 'Bulk-Climate-Global-Measures-CDP-v1-Csv-Delta-2023-12-10T17:03:07.473Z', 'bucketName': 'bulk-climate', 'packageId': '45ad-ee4b-9dfd4ed0-b6a2-f5a1b864f580', 'attributes': [{'name': 'ResultCount', 'value': '1670045'}, {'name': 'ContentType', 'value': 'Climate'}], 'files': ['4516-77c0-e8ce24de-a2a5-15bf070188d8'], 'numFiles': 1, 'contentFrom': '2023-12-03T16:55:00Z', 'contentTo': '2023-12-10T16:55:00Z', 'availableFrom': '2023-12-10T17:14:23Z', 'availableTo': '2023-12-24T17:14:22Z', 'status': 'READY', 'created': '2023-12-10T17:14:23Z', 'modified': '2023-12-10T17:14:44Z'}, {'id': '405e-0a82-285c6b92-8b4a-31936cf72fd3', 'name': 'Bulk-Climate-Global-Measures-WealthPro-CDP-Limited-v1-Csv-Init-2023-12-17T17:27:36.192Z', 'bucketName': 'bulk-climate', 'packageId': '436d-2891-52a25ccf-8d36-bc39f94b66ce', 'attributes': [{'name': 'ContentType', 'value': 'Climate'}, {'name': 'ResultCount', 'va

In [9]:
# lookup the field to datatype mapping from the field glossary document
fields = {
    'OrganizationId': 'DatapointValue',
    'FinancialPeriodFiscalYear': 'DatapointValue',
    'Scope1EstimatedTotal': 'DatapointValue',
    'Scope2EstimatedTotal': 'DatapointValue',
    'Scope3EstimatedUpstreamTotal': 'DatapointValue',
    'Scope3EstimatedDownstreamTotal': 'DatapointValue',
    'Scope1EstimatedMethod': 'DatapointValueText',
    'Scope2EstimatedMethod': 'DatapointValueText',
    'Scope3EstimatedUpstreamMethod': 'DatapointValueText',
    'Scope3EstimatedDownstreamMethod': 'DatapointValueText',
    'SalesinUSD': 'DatapointValue',
    'ExchangeRatetoUSD': 'DatapointValue',
    'EnterpriseValueincludingCashandShortTermInvestmentsinUSD': 'DatapointValue'
}

In [10]:
# parse out the entries in the file
climateData = []
matchFields = list(fields.keys())

for cMeasuresFile in bFiles:
    for l in cMeasuresFile.splitlines():
        jObj = json.loads(l)
        dt = {}
        for measure in jObj['ESGMeasureValue']['EsgMeasureValues']:
            if measure['EsgDataMeasure'] in matchFields:
                dt[measure['EsgDataMeasure']] = measure['EsgDatapointValue'][fields[measure['EsgDataMeasure']]]
            
        dt['OrganizationId'] = jObj['ESGStatementDetails']['OrganizationId']
        dt['FinancialPeriodFiscalYear'] = jObj['ESGStatementDetails']['FinancialPeriodFiscalYear']

        climateData.append(dt)

print(f'Loaded climate data for {len(climateData)} organizations')        

Loaded climate data for 114622 organizations


In [11]:
# load the dataset into a pandas dataframe
df1 = pd.DataFrame(climateData)
# save into a pickle
df1.to_pickle('ClimateData.pkl')